In [1]:
########################################
# NOTEBOOK: Online Factorization Machine with River
########################################

# %% [markdown]
# # 0. Setup & Data Loading
# 
# This notebook assumes you've already produced a `final_df` DataFrame with the columns:
# - `ClientID`, `ProductID`
# - `SalesNetAmountEuro`, `Quantity_sold`
# - Time-based columns: `TransactionDate`, `Month`, `DayOfWeek`, `Season`, etc.
# - Demographics: `ClientGender`, `Age`, `ClientSegment`, `ClientCountry`, ...
# - Product attributes: `Category`, `FamilyLevel1`, `FamilyLevel2`, `Brand`, `Universe`, ...
# - Past features (e.g. `CumulativeSpent`, `MostBoughtBrandSoFar`, etc.)
# - RFM windows: `Frequency_30`, `Monetary_30`, `Recency_30`, etc. (optional if you want them as features)
# 
# We'll simulate online training in chronological order.


In [3]:

# %%
import pandas as pd
import numpy as np
import random

# For the recommendation model
from river import compose
from river import reco
from river import metrics
from river import preprocessing

import tqdm



## 0.1 Load the Preprocessed Data

In [5]:

# %%
# Load final_df from parquet (or CSV, if you prefer)
final_df = pd.read_parquet("final_df.parquet")

# Verify the structure
print("Shape:", final_df.shape)
display(final_df.head())
display(final_df.dtypes)

# Sort by time so we can simulate incremental learning in chronological order
final_df = final_df.sort_values("TransactionDate").reset_index(drop=True)

final_df


Shape: (896963, 58)


,ClientID,ProductID,TransactionDate,StoreID,StoreCountry,Category,FamilyLevel1,FamilyLevel2,Universe,ClientSegment,...,Rolling90Pct_Cricket,Rolling90Pct_Beach,Rolling90Pct_Basketball,Rolling90Pct_Rugby,Rolling90Pct_Golf,Rolling90Pct_Softball,Rolling90Pct_Cycling,Rolling90Pct_Volleyball,Rolling90Pct_Running,Rolling90Pct_Skiing
0,4388436561084682799,3260004767786243986,2023-01-01,1913433680162801979,USA,Basketball,Ball,Wilson Evolution Basketball,Women,LOYAL,...,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,5475934562856106533,4081002095016762501,2023-01-01,7704230050291051317,GBR,Baseball,Ball,Wilson A1030,Men,LOYAL,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7571493122530801912,6392464777854173474,2023-01-01,6490882301505443815,USA,Football,Ball,Puma Final 1,Women,LOYAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9054036776577299596,4007202900314609762,2023-01-01,8616445013777361432,USA,Cycling,Bike,Specialized S-Works Roubaix,Men,LOYAL,...,NaN,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaN
4,7828763863563966653,1064014581685647413,2023-01-01,7215875119995113089,AUS,Tennis,Racket,Wilson Pro Staff RF97,Men,LOYAL,...,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN


ClientID                                     int64
ProductID                                    int64
TransactionDate                     datetime64[ns]
StoreID                                      int64
StoreCountry                                object
Category                                    object
FamilyLevel1                                object
FamilyLevel2                                object
Universe                                    object
ClientSegment                               object
ClientGender                                object
Age                                        float64
ClientCountry                               object
ClientOptINEmail                             int64
ClientOptINPhone                             int64
Quantity_sold                                int64
SalesNetAmountEuro                         float64
product_avg_price_order                    float64
avg_price                                  float64
Weekday                        

,ClientID,ProductID,TransactionDate,StoreID,StoreCountry,Category,FamilyLevel1,FamilyLevel2,Universe,ClientSegment,...,Rolling90Pct_Cricket,Rolling90Pct_Beach,Rolling90Pct_Basketball,Rolling90Pct_Rugby,Rolling90Pct_Golf,Rolling90Pct_Softball,Rolling90Pct_Cycling,Rolling90Pct_Volleyball,Rolling90Pct_Running,Rolling90Pct_Skiing
0,4388436561084682799,3260004767786243986,2023-01-01,1913433680162801979,USA,Basketball,Ball,Wilson Evolution Basketball,Women,LOYAL,...,NaN,NaN,1.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,569442104806296029,8689249957301923419,2023-01-01,6138815706411501953,AUS,Football,Jersey,Adidas Home Jersey,Women,LOYAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1124647793993972767,8822915206695862777,2023-01-01,5906069635056395270,FRA,Football,Ball,Adidas Telstar 18,Women,LOYAL,...,0.0,NaN,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2859127213945973734,7221725680641695356,2023-01-01,1172561306716917447,ARE,Handball,Ball,Select Ultimate,Men,LOYAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7896774237505151188,1527077913232062118,2023-01-01,6332090558343235680,FRA,Handball,Ball,Molten H3X5001,Women,LOYAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896958,6797242701886787550,1044719408392782437,2025-02-15,2828939986729367331,FRA,Hockey,Helmet,Bauer RE-AKT,Women,LOYAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
896959,1974857103601720390,4276734838749825428,2025-02-15,1172561306716917447,ARE,Tennis,Ball,Penn Championship,Women,LOYAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
896960,3501418887043844736,1923991482221095632,2025-02-15,6056406535832467469,USA,Cycling,Bike,Specialized S-Works Roubaix,Women,LOYAL,...,NaN,NaN,0.579672,NaN,NaN,NaN,0.202938,NaN,NaN,NaN
896961,8361760437617137777,4423794612747377145,2025-02-15,5766738857292149683,USA,Rugby,Jersey,Adidas Performance,Women,LOYAL,...,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN


In [ ]:

# %% [markdown]
# # 1. Feature Preparation for River
# 
# River’s **Factorization Machine** (`FMRecommender`) or `BiasedMF` expects:
# - A **user ID** feature
# - An **item ID** feature
# - (Optionally) additional side features
# - A **target** (for implicit, often 1 for a purchase, 0 for a negative sample)
# 
# We will:
# 1. Identify the user column (`ClientID`) and the item column (`ProductID`).
# 2. Prepare any extra features as a dictionary per row (e.g. `{"Age": 35, "Brand": "Nike"}`).
# 3. Decide how to handle **negative sampling**, because we only have positives (purchases).

# %% [markdown]
# ## 1.1 Basic negative sampling function
# 
# In implicit feedback scenarios, we typically have “user purchased item X” = positive, but no direct negative entries.  
# We can sample some “unpurchased” items as negatives (0).  
# 
# **Note**: For large catalogs, you might want to limit random sampling to a smaller set, or do popularity-based sampling. This code is just a simple illustration.


In [ ]:

# %%
all_items = final_df["ProductID"].unique().tolist()
all_items_set = set(all_items)

def sample_negative_items(user_item_pairs, n_neg=3):
    """
    For each (user, item) in user_item_pairs, 
    returns a list of (user, item, 1) + (user, some other item, 0) with n_neg negative items.
    """
    # user_item_pairs is a list/tuple of (user, item, features)
    # We'll sample n_neg distinct item(s) that user didn't buy for each positive.
    # Return a list of (u, i, features, y) for positives and negatives.
    results = []
    for (u, i, feats) in user_item_pairs:
        # Positive
        results.append((u, i, feats, 1.0))
        
        # Negative sampling
        # randomly pick n_neg items that are NOT i
        neg_samples = 0
        tries = 0
        while neg_samples < n_neg and tries < 100:  # safeguard
            candidate = random.choice(all_items)
            if candidate != i:
                # Build negative features as well
                neg_feats = feats.copy()  # you might also want to remove item-specific features, etc.
                # We'll just keep the user-level features, but we must keep in mind the item is different
                results.append((u, candidate, neg_feats, 0.0))
                neg_samples += 1
            tries += 1

    return results


In [ ]:

# %% [markdown]
# # 2. Defining a Ranking Metric: “Average Rank of Purchased Item”
# 
# We want to measure: **“On average, at what position in the ranked list does the actually purchased item appear?”**  
# 
# - A rank of `1` is best (the model’s top recommendation).  
# - Larger rank means the purchased item was lower in the list.  
# - We can keep a running average across all transactions.
# 
# **Naïve approach** to get the rank:
# 1. For a given transaction `(u, i)`, we compute a model score for *all candidate items* for user `u`.  
# 2. Sort items by descending predicted score.  
# 3. The rank is the 1-based index of the purchased item `i`.  
# 
# **Warning**: Doing this for **every** transaction with **all** items is expensive if you have a large catalog. You might only rank a subset or use approximate methods in production.  
# 
# For demonstration, we’ll implement a straightforward approach.


In [ ]:

# %%
class AverageRank:
    """
    Online metric for average rank of the purchased item.
    We'll track sum_of_ranks / count_of_events.
    """
    def __init__(self):
        self.sum_of_ranks = 0.0
        self.count = 0
        
    def update(self, rank):
        self.sum_of_ranks += rank
        self.count += 1
    
    def get(self):
        return self.sum_of_ranks / self.count if self.count > 0 else None
    
    def __repr__(self):
        val = self.get()
        return f"AverageRank={val:.3f}" if val else "AverageRank=None"

def get_rank(model, user_id, item_id, all_items_list, features):
    """
    Scores all items for this user, returns the 1-based rank of `item_id`.
    - `model`: River recommendation model with `predict_one(user, item, context_dict)` method.
    - `user_id`: the ID of the user
    - `item_id`: the ID of the purchased item
    - `all_items_list`: list of all item IDs (caution: large if your item catalog is huge)
    - `features`: dict of side features for the user or context
    """
    # Score each item
    scores = {}
    for it in all_items_list:
        # We'll augment features with "ItemID" so that the model can treat item as a feature
        # or we can rely on River's "FMRecommender" usage: model.predict_one(user, item, context)
        # This depends on how we define the pipeline below. We'll do a simpler version:
        scores[it] = model.predict_one({"user": user_id, "item": it, **features})
    
    # Sort items by descending score
    ranked_items = sorted(scores, key=scores.get, reverse=True)
    
    # Rank is index of item_id + 1
    rank_of_purchased = ranked_items.index(item_id) + 1
    return rank_of_purchased


In [ ]:

# %% [markdown]
# # 3. Building a River “FMRecommender” Pipeline
# 
# River has **two** main ways to do factorization-based recommendations:
# 1. `river.reco.BiasedMF` (a simpler matrix factorization approach)
# 2. `river.reco.FMRecommender` (Factorization Machine that can handle side features more gracefully)
# 
# We’ll use **`FMRecommender`** below because you have many side features (demographics, brand, RFM, etc.).  
# 
# **Key Points**:
# - We must define how user ID, item ID, and side features are passed to the model.  
# - `FMRecommender` expects a dictionary with keys named by default `"user"` and `"item"` (unless changed).  
# - We can include additional features in that dictionary.  
# - The target `y` will be `1` for positive, `0` for negative.


In [ ]:

# %%
# Let's define our model
model = reco.FMRecommender(
    n_factors=10,             # Dimensionality of the latent factors
    intercept=True, 
    seed=42,
    optimizer=optim=preprocessing.StandardScaler() # not typical here, 
    # but let's keep it simpler: we can just do the default Adam or SGD
    # For example:
    # optimizer=optim.SGD(0.01)
)


In [ ]:

# Alternatively:
from river import optim
model = reco.FMRecommender(
    n_factors=10,
    intercept=True,
    seed=42,
    optimizer=optim.SGD(learning_rate=0.01),  # try a small LR
    loss=optim.loss.BPRLoss(),               # BPR pairwise ranking loss (if you want a ranking objective)
)


In [ ]:

# We can wrap the model with a pipeline that does feature transformations if needed, e.g., one-hot encoding
# But FMRecommender can handle categoricals automatically if we pass them as strings.

# For example:
# pipeline = compose.Pipeline(
#     model
# )
# We'll just use `model` directly in this example.


In [ ]:

# %% [markdown]
# # 4. Online Training Loop with Negative Sampling and Average Rank
# 
# We’ll step through `final_df` in chronological order. For each row (transaction) we do:
# 
# 1. Extract `(user, item, side_features, rating=1)`.  
# 2. **Before** we update the model, we can measure the rank of the purchased item (if the user + item are not brand new).  
#    - This simulates “the model’s knowledge up to the previous transaction.”  
# 3. Generate negative samples for that user (0 rating).  
# 4. Update the model with the positive and negative samples.  
# 5. Keep track of the running average rank metric.  
# 
# We’ll do a short loop (e.g., first 5,000 or 10,000 transactions) to keep this example from being too slow if the dataset is large.  
# 
# **Caution**: 
# - The naive approach `get_rank` calls the model’s `predict_one` for **all items** each time. That can be extremely slow for big catalogs.  
# - In production, consider approximations or smaller candidate sets.


In [ ]:

# %%
# We create an instance of our AverageRank metric
avg_rank_metric = AverageRank()

# We define how many rows we want to process for the demo
# You can do all rows, but it might take a very long time if the dataset is huge
MAX_ROWS = 5000  

processed = 0

# We'll store some logs
log_step = 1000

for idx, row in final_df.iterrows():
    # Limit to a subset for demonstration
    if processed >= MAX_ROWS:
        break
    processed += 1
    
    user_id = str(row["ClientID"])  # Convert to string for consistency in FM
    item_id = str(row["ProductID"])
    
    # Build side_features dict. You can include whichever columns are relevant.
    # For example:
    side_feats = {
        "ClientGender": str(row["ClientGender"]) if pd.notnull(row["ClientGender"]) else "Unknown",
        "ClientSegment": str(row["ClientSegment"]) if pd.notnull(row["ClientSegment"]) else "Unknown",
        "Brand": str(row["Brand"]) if pd.notnull(row["Brand"]) else "Unknown",
        "Category": str(row["Category"]) if pd.notnull(row["Category"]) else "Unknown",
        "Universe": str(row["Universe"]) if pd.notnull(row["Universe"]) else "Unknown",
        # Numeric features (careful to keep them numeric or string):
        "Age": row["Age"] if pd.notnull(row["Age"]) else 0,
        "CumulativeSpent": row["CumulativeSpent"],
        "DayOfWeek": str(row["DayOfWeek"]),
        "Season": str(row["Season"]),
        # etc. Add RFM or other fields if you want:
        "Frequency_30": row["Frequency_30"],
        "Recency_30": row["Recency_30"]
        # ...
    }
    
    # 1) Evaluate the rank *before* we train on this new (user, item) => "test on the fly"
    #    Only do this if the model has seen some data already (i.e., after the first 100 or so)
    if processed > 100:  # skip the very first interactions
        rank = get_rank(model, user_id, item_id, all_items, side_feats)
        avg_rank_metric.update(rank)
    
    # 2) Prepare the positive sample
    # The model expects a dictionary with "user" and "item" keys by default
    x_pos = {"user": user_id, "item": item_id}
    # Also add side feats
    x_pos.update(side_feats)
    
    # 3) Negative sampling for this user
    # We'll pass them in a batch. Let's generate them:
    # We'll do 3 negative items for demonstration
    negative_tuples = []
    n_neg = 3
    negatives = set()
    tries = 0
    while len(negatives) < n_neg and tries < 50:
        candidate = random.choice(all_items)
        if candidate != item_id:
            negatives.add(candidate)
        tries += 1
    
    # 4) Update the model with the positive example
    # BPR loss typically needs a pairwise update, but we can do an approximation:
    model = model.learn_one(x_pos, 1.0)
    
    # 5) Update the model with negative examples
    for neg_item in negatives:
        x_neg = {"user": user_id, "item": neg_item}
        x_neg.update(side_feats)
        model = model.learn_one(x_neg, 0.0)
    
    if processed % log_step == 0:
        print(f"Processed {processed} transactions, current {avg_rank_metric}")

print(f"Final Average Rank on the last {processed - 100} transactions: {avg_rank_metric}")